In [2]:
import re
import os
import numpy as np
import pandas as pd
from pandas import DataFrame as df

au2kcal = 627.50957099203276

In [3]:
def W1_method(table,ZPE):
    
   #**************************
    SCF_DZ=table['ESCF'][0]
    SCF_TZ=table['ESCF'][1]
    SCF_QZ=table['ESCF'][2]
    CCSD_DZ=table['ECCSD'][0]
    CCSD_TZ=table['ECCSD'][1]
    CCSD_QZ=table['ECCSD'][2]
    CCSDT_DZ=table['ECCSDT'][0]
    CCSDT_TZ=table['ECCSDT'][1]
    FC_MTsmall=table['ECCSDT'][3]
    FULL_MTsmall=table['ECCSDT'][4]
    
  #*****************************
        #*****************************
    CCSD_CORR_DZ=CCSD_DZ-SCF_DZ
    CCSD_CORR_TZ=CCSD_TZ-SCF_TZ
    CCSD_CORR_QZ=CCSD_QZ-SCF_QZ
    
    CCSDT_CORR_DZ=CCSDT_DZ-CCSD_DZ
    CCSDT_CORR_TZ=CCSDT_TZ-CCSD_TZ
    #******************************
    SCF_temp = (SCF_QZ-SCF_TZ)/(((4/3)**5)-1)
    SCF_INF= SCF_QZ+SCF_temp
    
    CCSD_temp = (CCSD_CORR_QZ-CCSD_CORR_TZ)/(((4/3)**(3.22))-1)
    CCSD_INF = CCSD_CORR_QZ+CCSD_temp
    
    CCSDT_temp = (CCSDT_CORR_TZ-CCSDT_CORR_DZ)/(((3/2)**(3.22))-1)
    CCSDT_INF = CCSDT_CORR_TZ+CCSDT_temp
    
    CORE_CORR = FULL_MTsmall-FC_MTsmall
    
    ZPE_CORR = ZPE*0.985
    
    W1_electronic = SCF_INF+CCSD_INF+CCSDT_INF+CORE_CORR
    W1_0K = W1_electronic+ZPE_CORR
    #print("*************************************************")
    """
    print("Extrapolated_SCF is {}".format(SCF_INF))
    print("Extrapolated_CCSD is {}".format(CCSD_INF))
    print("Extrapolated_CCSDT is {}".format(CCSDT_INF))
    print("CORE Component is {}".format(CORE_CORR))
    print("W1 Electronic Energy is {}".format(W1_electronic))
     """
    #print("W1(0K) is {}".format(W1_0K))
    #print("***************************************************")
    return W1_0K

def make_df(data):
    values=data 
    column = ['ESCF',
             'ECCSD',
             'ECCSDT']
    row = ['VDZ','VTZ','VQZ','MTsmall','MTsmall_full']
    return pd.DataFrame(values,row,column)


## Definitions of Molecules in log files
* Mol_01 : R
* Mol_02 : ROO
* Mol_03 : QOOH
* Mol_04 : cy-Ether
* Mol_05 : Alkene

 ## Molpro W1 Results for reaction energies

- For a particular system, Molpro outupt files are stored in '../SYSTEM_NAME/SYSTEM_NAME_W1_FILES'.<br>  
   For example: '../ETHYL/ETHYL_W1_FILES'<br>  
   

-  This script works in three steps:<br>  

   (1) Extract SCF, CCSD and CCSD(T) energy for VDZ,VTZ,VQZ,MTsmall, and MTsmall_full steps, for all molecules of given systems,and store them as a CSV file in  '../SYSTEM_NAME/SYSTEM_NAME_DATA/'  dir. For each molecule one csv file is generate. Unit for this table is <mark>hartree.</mark>

        Name of this csv file is: SYSTEM_NAME_Mol_NAME_Molpro.csv .

       Example: Inside 'ETHYL/ETHYL_DATA' dir,   ETHYL_Mol_01_molpro.csv, ETHYL_Mol_02_molpro.csv, ETHYL_Mol_03_molpro.csv, ETHYL_Mol_04_molpro.csv and ETHYL_Mol_05_molpro.csv are stored. 
    ![Example_Molpro.png](attachment:Example_Molpro.png)

(2) In next step, basis set extrapolation is employed according to the W1 method. Final W1 energies for a given system are stored in '../CSV_FILES/' dir .  Unit for this table is <mark>hartree.</mark>
   
     Name of the csv file for a given system is: SYSTEM_NAME_W1.csv.
   
    For example: For ETHYL sytem, file name is : ETHYL_W1.csv 
![Example_W1.png](attachment:Example_W1.png)

(3) In final step, Energies of ROO, QOOH, Alkene+OH, cy-Ether+OOH are calculated with respect to R+O2.

For each system, results are stoed in '../CSV_FILES' dir. Unit for this table is <mark>kcal/mol.</mark>

     Name of the csv file for a given system is : SYSTEM_NAME_W1_Rxn_E.csv.

    For example : For ETHYL system, file name is : ETHYL_W1_Rxn_E.csv

![Example_W1_Rxn.png](attachment:Example_W1_Rxn.png)

## Script

In [8]:
DIR_OUTPUT = '../CSV_FILES/'

SYSTEMS = ['ETHYL','ISOPROPYL','ISOBUTYL','TERTBUTYL','NEOPENTYL','CYCLOHEXYL','CYCLOHEXENYL',
           'CYCLOHEXADIENYL']
#SYSTEMS = ['ETHYL']
MOLECULE = ['Mol_01','Mol_02','Mol_03','Mol_04','Mol_05']

METHOD = ['VDZ','VTZ','VQZ','MTsmall','MTsmall_full']
SYSTEMS_DF = []
for isystem in SYSTEMS:
    
    DIR_DATA = '../'+isystem+'/'+isystem+'_DATA/'
    ZPE_MOL = np.loadtxt(DIR_DATA+isystem+'_ZPE.dat')
    #Hcorr = np.loadtxt(DIR_DATA+isystem+'_Hcorr.dat')

    DIR_OUT_FILES = '../'+isystem+'/'+isystem+'_W1_FILES/'
    MOL_DF = []
    W1_DF = pd.DataFrame(np.zeros((len(MOLECULE),1)),MOLECULE,columns=['W1'])
    for imol in MOLECULE:
        INIT_DF = make_df(np.zeros((5,3)))
        for imethod in METHOD:
        
            FILENAME = DIR_OUT_FILES+imol+'_'+imethod+'.out'
            file = open(FILENAME,"r")
            #print(FILENAME)

            for line in file:
                if re.search('Reference energy',line):
                    #print(line)
                    INIT_DF['ESCF'][imethod]=float(line.split()[2])
                
                if re.search('RHF-UCCSD energy',line):
                    INIT_DF['ECCSD'][imethod] = float(line.split()[2])
                
                if re.search('!RHF-UCCSD\(T\) energy',line):
                    INIT_DF['ECCSDT'][imethod] = float(line.split()[2])
                
                if re.search("CCSD total energy",line):
                    INIT_DF['ECCSD'][imethod] = float(line.split()[3])
                
                if re.search("!CCSD\(T\) total energy",line):
                    INIT_DF['ECCSDT'][imethod] = float(line.split()[3])
        
        INIT_DF.index.name='BASIS'
        INIT_DF.to_csv(DIR_DATA+isystem+'_'+imol+'_Molpro.csv')
        INIT_DF.to_csv(DIR_DATA+imol+'_Molpro.csv')  
        W1_method(INIT_DF,ZPE_MOL[MOLECULE.index(imol)])
        W1_DF['W1'][imol] =  W1_method(INIT_DF,ZPE_MOL[MOLECULE.index(imol)])
        MOL_DF.append(INIT_DF)
    #print(W1_DF)
    #print(DIR_OUTPUT+isystem+'_W1.csv')
    W1_DF.index.name='MOLECULE'
    W1_DF.to_csv(DIR_OUTPUT+isystem+'_W1.csv')
    SYSTEMS_DF.append(MOL_DF)


/tmp/ipykernel_11703/704149334.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_DZ=table['ESCF'][0]
/tmp/ipykernel_11703/704149334.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_TZ=table['ESCF'][1]
/tmp/ipykernel_11703/704149334.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  SCF_QZ=table['ESCF'][2]
/tmp/ipykernel_11703/704149334.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In

In [9]:
SYSTEMS = ['ETHYL','ISOPROPYL','ISOBUTYL','TERTBUTYL','NEOPENTYL','CYCLOHEXYL','CYCLOHEXENYL',
           'CYCLOHEXADIENYL']
DATA_1 = pd.read_csv(DIR_OUTPUT+'O2_OH_OOH'+'_W1.csv',index_col='MOLECULE')
for isystem in SYSTEMS:
    DATA_2 = pd.read_csv(DIR_OUTPUT+isystem+'_W1.csv',index_col='MOLECULE')
    NAME_SPECIES = ['R+O2','ROO','QOOH','cy-Ether+OH','Alkene+OOH']
    W1_Rxn_DF = pd.DataFrame(np.zeros((len(NAME_SPECIES),1)),NAME_SPECIES,['W1'])
    E_ref = DATA_2['W1']['Mol_01']+DATA_1['W1']['O2']
    W1_Rxn_DF['W1']['ROO'] = DATA_2['W1']['Mol_02']-E_ref
    W1_Rxn_DF['W1']['QOOH'] = DATA_2['W1']['Mol_03']-E_ref
    W1_Rxn_DF['W1']['cy-Ether+OH'] = DATA_2['W1']['Mol_04']+DATA_1['W1']['OH']-E_ref
    W1_Rxn_DF['W1']['Alkene+OOH'] = DATA_2['W1']['Mol_05']+DATA_1['W1']['OOH']-E_ref
    W1_Rxn_DF= W1_Rxn_DF*au2kcal
    W1_Rxn_DF.index.name='SPECIES'
    W1_Rxn_DF.to_csv(DIR_OUTPUT+isystem+'_W1_Rxn_E.csv')

In [10]:
DATA = pd.read_csv('../ETHYL/ETHYL_DATA/ETHYL_'+'Mol_01'+'_Molpro.csv',index_col='BASIS')
DATA

,ESCF,ECCSD,ECCSDT
BASIS,,,
VDZ,-78.601439,-78.918804,-78.927202
VTZ,-78.623490,-78.993829,-79.006371
VQZ,-78.628337,-79.013602,-79.027125
MTsmall,-78.623792,-78.998654,-79.011407
MTsmall_full,-78.653060,-79.127593,-79.141152


In [13]:
DATA = pd.read_csv(DIR_OUTPUT+'ETHYL'+'_W1_Rxn_E.csv',index_col='SPECIES')
DATA

,W1
SPECIES,
R+O2,0.000000
ROO,-32.815238
QOOH,-16.196410
cy-Ether+OH,-32.580178
Alkene+OOH,-13.289945
